In [1]:
import os

In [2]:
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

from keras.layers import Input, Dense 
from keras.models import Model

In [3]:

is_init = False
labels = []
label_dict = {}
c = 0

for file in sorted(os.listdir()):
    if file.endswith(".npy") and file != "labels.npy":

        data = np.load(file)
        current_size = data.shape[0]
        class_name = file.split(".")[0]

        if not is_init:
            X = data
            y = np.array([class_name] * current_size).reshape(-1, 1)
            is_init = True
        else:
            X = np.concatenate((X, data))
            y = np.concatenate(
                (y, np.array([class_name] * current_size).reshape(-1, 1))
            )

        labels.append(class_name)
        label_dict[class_name] = c
        c += 1

# -------------------------------
# LABEL ENCODING
# -------------------------------
for i in range(len(y)):
    y[i, 0] = label_dict[y[i, 0]]

y = np.array(y, dtype="int32")
y = to_categorical(y)

# -------------------------------
# SHUFFLE DATA (IMPORTANT)
# -------------------------------
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

# -------------------------------
# MODEL
# -------------------------------
ip = Input(shape=(X.shape[1],))
x = Dense(512, activation="relu")(ip)
x = Dense(256, activation="relu")(x)
op = Dense(y.shape[1], activation="softmax")(x)

model = Model(inputs=ip, outputs=op)

model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(X, y, epochs=50, batch_size=32)

# -------------------------------
# SAVE MODEL & LABELS
# -------------------------------
model.save("model.h5")
np.save("labels.npy", np.array(labels))

print("✅ Training completed successfully")

Epoch 1/50
29/29 [==============================] - 1s 10ms/step - loss: 2.0327 - accuracy: 0.2533
Epoch 2/50
29/29 [==============================] - 0s 9ms/step - loss: 1.5622 - accuracy: 0.4778
Epoch 3/50
29/29 [==============================] - 0s 10ms/step - loss: 1.1813 - accuracy: 0.6311
Epoch 4/50
29/29 [==============================] - 0s 9ms/step - loss: 0.9695 - accuracy: 0.7156
Epoch 5/50
29/29 [==============================] - 0s 9ms/step - loss: 0.8237 - accuracy: 0.7356
Epoch 6/50
29/29 [==============================] - 0s 10ms/step - loss: 0.7125 - accuracy: 0.8044
Epoch 7/50
29/29 [==============================] - 0s 9ms/step - loss: 0.6664 - accuracy: 0.8156
Epoch 8/50
29/29 [==============================] - 0s 9ms/step - loss: 0.6106 - accuracy: 0.8089
Epoch 9/50
29/29 [==============================] - 0s 9ms/step - loss: 0.5803 - accuracy: 0.8178
Epoch 10/50
29/29 [==============================] - 0s 9ms/step - loss: 0.5430 - accuracy: 0.8389
Epoch 11/50
29/2

In [4]:
print(X)
print(y)
print("X shape:", X.shape)
print("y shape:", y.shape)

print("First 5 X samples:\n", X[:5])
print("First 5 y samples:\n", y[:5])


[[-0.00391307  0.04996538  0.         ...  0.          0.
   0.        ]
 [-0.00502065  0.06180966  0.         ...  0.          0.
   0.        ]
 [-0.00060552  0.05093163  0.         ...  0.          0.
   0.        ]
 ...
 [-0.00058916  0.05183429  0.         ...  0.          0.
   0.        ]
 [ 0.00194409  0.03760421  0.         ...  0.          0.
   0.        ]
 [-0.00039646  0.05238926  0.         ...  0.          0.
   0.        ]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
X shape: (900, 1020)
y shape: (900, 9)
First 5 X samples:
 [[-0.00391307  0.04996538  0.         ...  0.          0.
   0.        ]
 [-0.00502065  0.06180966  0.         ...  0.          0.
   0.        ]
 [-0.00060552  0.05093163  0.         ...  0.          0.
   0.        ]
 [-0.00077486  0.0512597   0.         ...  0.          0.
   0.        ]
 [-0.00071722  0.04493266  0.         ...  0.    